In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
import numpy as np
import os
from Building import Building
#from Agent import Agent
import time
#====================================================================================


#====================================================================================
#Building Setting
lift_num = 2
buliding_height = 10
max_people_in_floor = 10

add_people_at_step = 25
add_people_prob = 0.8

#Create building with 4 elevators, height 10, max people 30 in each floor
building = Building(lift_num, buliding_height, max_people_in_floor)

#Agent controls each elevator
#agent = Agent(buliding_height, lift_num, 4)
#agent.reload(280)
#The goal is to bring down all the people in the building to the ground floor

epochs = 1000
max_steps = 100
global_step = 0
T_horizon = 20
reward_list = []
print_interval = 20

In [3]:
reward_list = []

In [4]:
#Hyperparameters
learning_rate = 0.0001
gamma         = 0.99
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 3
T_horizon     = 20
class Agent(nn.Module):
    def __init__(self, state_dim,elevator_num,action_dim,learning_rate):
        self.state_dim = state_dim
        self.elevator_num = elevator_num
        self.action_dim = action_dim
        
        super(Agent,self).__init__()
        self.memory = []

        self.fc1 = nn.Linear(self.state_dim,256)
        self.fc2 = nn.Linear(256,256)
        self.policy = nn.ModuleList([nn.Linear(256, self.action_dim) for _ in range(self.elevator_num)])
        #self.policy = [nn.Linear(256, self.action_dim) for x in range(elevator_num)]
        self.value = nn.Linear(256, 1)
        self.optimizer = optim.Adam(self.parameters(),lr = learning_rate)
        
    def get_action(self,x, softmax_dim = 0):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        xs = [layer(x) for layer in self.policy]
        x = [F.softmax(x,dim = softmax_dim) for x in xs]
        return x

    
    def get_value(self,x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.value(x)
        return x
    
    def put_data(self,data):
        self.memory.append(data)
        
    def make_batch(self):
        state_list, action_list, reward_list, next_state_list, prob_list, done_list = [],[],[],[],[],[]
        for data in self.memory:
            state,action,reward,next_state,prob,done = data
            state_list.append(state)
            action_list.append([action])
            reward_list.append([reward])
            prob_list.append([prob])
            next_state_list.append(next_state)
            done_mask = 0 if done else 1
            done_list.append([done_mask])
        self.memory = []
        
        s,a,r,next_s,done_mask,prob = torch.tensor(state_list,dtype=torch.float),\
                                        torch.tensor(action_list),torch.tensor(reward_list),\
                                        torch.tensor(next_state_list,dtype=torch.float),\
                                        torch.tensor(done_list,dtype = torch.float),\
                                        torch.tensor(prob_list)
        return s,a,r,next_s,done_mask,prob
    
    def train(self):
        state,action,reward, next_state,done_mask,action_prob = self.make_batch()
        
        for i in range(K_epoch):
            td_error = reward + gamma * self.get_value(next_state) * done_mask
            delta = td_error - self.get_value(state)
            delta = delta.detach().numpy()
            advantage_list = []
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_list.append([advantage])
            advantage_list.reverse()
            advantage = torch.tensor(advantage_list,dtype = torch.float)
            
            
            now_action = self.get_action(state,softmax_dim = 1)

            now_action = torch.stack(now_action)

            action_select = np.array([[x[0][idx] for x in action] for idx in range(self.elevator_num)])
            action_select = torch.from_numpy(action_select).reshape(self.elevator_num,-1,1)

            now_action = now_action.gather(2,action_select)
            
            action_prob = action_prob.reshape(-1,2,1)
            action_prob = action_prob.permute(1,0,2)
            ratio = torch.exp(torch.log(now_action) - torch.log(action_prob))
        

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio , 1-eps_clip, 1 + eps_clip) * advantage
            
            loss = - torch.min(surr1,surr2) + F.smooth_l1_loss(self.get_value(state),td_error.detach())
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
    

In [5]:
model = Agent(buliding_height+ lift_num *2,lift_num,4,learning_rate) #a = Agent(5,2,4,0.0001)


In [7]:
for epoch in range(epochs):
    building.empty_building()
    while building.target == 0 :
        building.generate_people(add_people_prob)
    state = building.get_state()
    done = False
    global_step = 0
    ave_reward = 0
    while not done:
        for t in range(T_horizon):
            global_step += 1
            if (global_step % 50 == 0) & global_step > 0 :
                pass
                #building.generate_people(add_people_prob/2)
            prev_people = building.get_arrived_people()
            action_prob = model.get_action(torch.from_numpy(np.array(state)).float())
            
            #m = Categorical(action_prob)
            m = [Categorical(x) for x in action_prob]
            action = [x.sample().item() for x in m]
            building.perform_action(action)
            reward = building.get_reward(prev_people) 
            
            next_state = building.get_state()
            if (building.get_arrived_people() == building.target) :
                done = True 
            #model.put_data((state, action, reward/100.0, next_state, np.array(action_prob)[action].item(), done))
            model.put_data((state, action, reward/100.0, next_state,\
                            [x[action[idx]].item() for idx,x in enumerate (action_prob)], done))
            state = next_state
            ave_reward += reward
            if done or global_step > 300:
                done = True
                break

        model.train()
    #print("Epoch: %d Step: %d Average Reward: %.4f"%(epoch, global_step, ave_reward/global_step))
    if epoch%print_interval==0 and epoch!=0:
        print("# of episode :{}, avg score : {:.1f}".format(epoch, global_step/print_interval))
    if (epoch % 100 == 0 )& (epoch != 0):
        torch.save(model.state_dict(), './model_weights/model_'+str(epoch))
    reward_list.append(global_step)

# of episode :20, avg score : 15.1
# of episode :40, avg score : 15.1
# of episode :60, avg score : 15.1
# of episode :80, avg score : 15.1
# of episode :100, avg score : 15.1
# of episode :120, avg score : 15.1
# of episode :140, avg score : 15.1
# of episode :160, avg score : 15.1
# of episode :180, avg score : 15.1
# of episode :200, avg score : 15.1
# of episode :220, avg score : 15.1
# of episode :240, avg score : 15.1
# of episode :260, avg score : 15.1


KeyboardInterrupt: 